# Elements Of Data Processing (2021S1) - Week 4

In [ ]:
# a way of ignoring errors
import warnings
warnings.simplefilter(action='ignore')

import matplotlib.pyplot as plt
import seaborn as sns

### Question 1
Consider the 1-dimensional data set with 10 data points {1,2,3,...10}. Show the iterations of the k-means algorithm using Euclidean distance when $k = 2$, and the random seeds are initialized to {1, 2}.

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

# usually we don't specify the random points as sklearn has a built-in method of setting fixed random states
initial_clusters = np.array([[1], [2]])
data_points = np.array([[i] for i in range(1, 11)])

kmean = KMeans(n_clusters=2, init=initial_clusters)
kmean.fit(data_points)

kmean.cluster_centers_, kmean.labels_

- Centroids at 2.5, 7.5
- All data points below 4 belong to 2.5, all data points above 5 belong to 7.5

### Question 2
Repeat Exercise 1 using agglomerative hierarchical clustering and Euclidean distance with single linkage (min) criterion.


In [ ]:
from scipy.spatial.distance import pdist, squareform

- `pdist` computes Pairwise Distance given some metric (i.e Euclidean)
- `squareform` displays the results as a matrix

In [ ]:
data_points = np.array([[i] for i in range(1, 11)])

distance = pdist(data_points, 'euclidean')

print(sns.heatmap(squareform(distance)))

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

- `linkage` performs the  hierarchical/agglomerative clustering.
- `method='single'` denotes our method - i.e choose the closest (min) point.
    - Other ones include `average` (UPGMA algorithm);
    - and `complete` (clusters are singletone, which are then sequentially combined into larger clusters until all elements end up being in the same cluster).
- `dendrogram` plots the clusters

Read More: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

In [ ]:
hc = linkage(distance, method='single')
dendrogram(hc, labels=data_points)

plt.show()

- As you can see, each point will only correspond to its neighbouring points (min distance away)

In [ ]:
hc2 = linkage(distance, method='complete')
dendrogram(hc2, labels=data_points)

plt.show()

## Linear Regression
- The bread and butter of most predictive models in industry.
- Learn more in Linear Statistical Models (MAST30025). We only very very briefly cover it in EoDP!
- The dataset is Boston house prices that comes with `sklearn`. Please see the description below:

In [ ]:
from sklearn import datasets 
import pandas as pd

data = datasets.load_boston()
print(data.DESCR)

For Linear Regression:
- You need an `X` matrix (aka Design Matrix) which are the values you use to *predict*.
- You also need a `y` matrix (aka Predictions) which are the values you want to *predict*.

In [ ]:
# Design Matrix
df = pd.DataFrame(data.data, columns=data.feature_names)
df.head()

In [ ]:
# Prediction (MEDV)
y = pd.DataFrame(data.target, columns=["MEDV"])
y.head()

## Example Linear Model
- Let's fit a regression model using two variables `RM` (average number of rooms per dwelling) and `LSTAT` (% of lower status in the population) to predict `MEDV` (Median value of owner-occupied homes in \$1000's)
- `train_test_split` used to split our data into training and testing. We do this as we need to *evaluate* our model on data it **has not seen before**.
- `mean_squared_error` (MSE) is a very common metric for evaluating our models' performance. It calculates the average error$^2$ and is used to compare two different models (useless on its own).
- `r2_score` (R$^2$) is another metric for evaluating the models performance (though it's actually not that good, adjusted R$^2$ is better).

#### Evaulation Metrics
- MSE: The lower the better
- R2: Between 0 and 1, where a high R2 indicates a better fit

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Create the design matrix using the two variables
X = df[['RM', 'LSTAT']]

In [ ]:
# split it so that we have 80% data for training (finding the coefficients of our model)
# and 20% for evaluating the model using MSE
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

In [ ]:
# train the model
lm = LinearRegression().fit(X_train, y_train)

In [ ]:
# predict values of y given our hidden test set
y_pred = lm.predict(X_test)

Let's compare the first few results

In [ ]:
y_test[:5], y_pred[:5]

As you can see, our model seems to be roughly predicting within a reasonable amount away from the true expected predictions.

In [ ]:
MSE = mean_squared_error(y_test, y_pred)
R2 = lm.score(X_test, y_test)
MSE, R2

- At the moment, MSE doesn't mean much as we don't have a model to compare it to;
- but the R$^2$ suggests our model isn't doing quite well.

## Question 4  
Write out the fitted linear model in Example 1:
- `alpha` is the intercept parameter.
- `beta` is the array of coefficients

In [ ]:
alpha = lm.intercept_
beta = lm.coef_

In [ ]:
alpha, beta

In [ ]:
RM = X['RM'].values
LSTAT = X['LSTAT'].values

# add code below
MEDV = ...

- What do the coefficients indicate?

## Question 5
Residuals:
- A residual is defined to be the difference between the observed value (true value) and the estimated value (our prediction).
- If our estimates are good, then the residuals should be very close to the true errors.
- For example, a model with perfect fit that can predict with 100% accuracy should have 0 residuals.

Interpret the residuals for the test and training data of the model in Example 1.

In [ ]:
# make predictions
y_pred_test = lm.predict(X_test)
y_pred_train = lm.predict(X_train)

# calculate residuals
residual_test = [true_val - estimated_val for true_val, estimated_val in zip(y_test, y_pred_test)]
residual_train = [true_val - estimated_val for true_val, estimated_val in zip(y_train, y_pred_train)]

# plot residuals
plt.scatter(y_pred_test, residual_test, label='R^2 test', color='red')
plt.scatter(y_pred_train, residual_train, label='R^2 train', alpha=0.15)

# plot the 0 line (we want our residuals close to 0)
plt.plot([min(y_pred_train), max(y_pred_train)], [0,0], color='green')

plt.legend()

plt.title("Residual plot for LM")
plt.show()

1. Do the points have large residuals (differences between true and estimated)?
    - We want small residuals that are close to 0.
1. Is there a trend or bias in the residuals (i.e does the residuals look evenly spread and flat)?
    - We don't want any bias or trend.
1. Is there a pattern or correlation in the residuals (i.e is there some kind of relationship in the residuals)?
    - We don't want any correlation.

## Question 6
Fit another linear model using all 13 variables to predict **MEDV**.
- Compare the results with those of the model in Question 1 (looking at MSE and residual plots)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.values, y.values, test_size=0.2, random_state=42)

lm_full = LinearRegression().fit(X_train, y_train)

y_pred = lm_full.predict(X_test)

MSE = mean_squared_error(y_test, y_pred)
R2 = lm_full.score(X_test, y_test)
MSE, R2

Now our MSE metric is much more useful.
- What's the difference?
- What does it imply?

Likewise with R$^2$. Is our model better?

In [ ]:
# make predictions
y_pred_test = lm_full.predict(X_test)
y_pred_train = lm_full.predict(X_train)

# calculate residuals
residual_test = [true_val - estimated_val for true_val, estimated_val in zip(y_test, y_pred_test)]
residual_train = [true_val - estimated_val for true_val, estimated_val in zip(y_train, y_pred_train)]

# plot residuals
plt.scatter(y_pred_test, residual_test, label='R^2 test', color='red')
plt.scatter(y_pred_train, residual_train, label='R^2 train', alpha=0.15)

# plot the 0 line (we want our residuals close to 0)
plt.plot([min(y_pred_test), max(y_pred_test)], [0,0], color='green')

plt.yticks(range(-20, 31, 10))

plt.legend()

plt.title("Residual plot for LM Full")
plt.show()

<img align="left" src="res.png">

- Do the plots look similar or a bit more different?